In [10]:
import os
import geopandas as gpd
from rasterio.plot import show
import rasterio
import pandas as pd

In [2]:
# update the wd path to be able to laod the module
os.chdir('../..') 
os.getcwd()

'C:\\Users\\Saif.Shabou\\OneDrive - World Resources Institute\\Documents\\cities-indicators-framework\\citymetrix\\cities-cif'

In [3]:
os.environ['GCS_BUCKET']='gee-exports'
os.environ['GOOGLE_APPLICATION_USER']='developers@citiesindicators.iam.gserviceaccount.com'
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='C:\\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\cities-indicators-framework\citymetrix\credentials-citiesindicators.json'

# Get boundaries

In [4]:
# load boundary of a region of interest
boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-BRA-Salvador-ADM4union.geojson'
city_gdf_aoi = gpd.read_file(boundary_path, driver='GeoJSON')
city_gdf_aoi.head()

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140..."


In [5]:
# load boundary of sub-georgaphy
boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-BRA-Salvador-ADM4.geojson'
city_gdf_sub = gpd.read_file(boundary_path, driver='GeoJSON')
city_gdf_sub.head()

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry
0,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.40125 -12.96457, -38.40126..."
1,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.39898 -12.96269, -38.39898..."
2,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.37346 -12.93345, -38.37386..."
3,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.41741 -12.97578, -38.41746..."
4,BRA-Salvador_ADM4_5,ADM4,Jardim Armação,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.43383 -12.98742, -38.43386..."


# Mean Tree Cover

In [6]:
from city_metrix import mean_tree_cover

Authenticating to GEE with configured credentials file.


In [7]:
%%time

# for one geography
city_gdf_aoi["mean_tree_cover"] = mean_tree_cover(city_gdf_aoi) 
city_gdf_aoi

CPU times: total: 8.16 s
Wall time: 38.1 s


,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry,mean_tree_cover
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140...",28.366773


In [8]:
%%time

# for sub geographies
city_gdf_sub["mean_tree_cover"] = mean_tree_cover(city_gdf_sub) 
city_gdf_sub.head()

CPU times: total: 7.39 s
Wall time: 38.4 s


,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry,mean_tree_cover
0,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.40125 -12.96457, -38.40126...",52.392370
1,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.39898 -12.96269, -38.39898...",43.309331
2,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.37346 -12.93345, -38.37386...",27.407306
3,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.41741 -12.97578, -38.41746...",15.772191
4,BRA-Salvador_ADM4_5,ADM4,Jardim Armação,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.43383 -12.98742, -38.43386...",21.087379


In [11]:
# Calculate indicator for multiple regions of interest

boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-BRA-Salvador-ADM4union.geojson'
city_Salvador = gpd.read_file(boundary_path, driver='GeoJSON')

boundary_path = 'https://cities-indicators.s3.eu-west-3.amazonaws.com/data/boundaries/boundary-IDN-Jakarta-ADM4union.geojson'
city_Jakarta = gpd.read_file(boundary_path, driver='GeoJSON')

cities = pd.concat([city_Salvador, city_Jakarta])
cities = cities.reset_index(drop=True)
cities

,geo_id,geo_level,geo_name,geo_parent_name,creation_date,geometry
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,2022-08-03,"MULTIPOLYGON (((-38.50135 -13.01134, -38.50140..."
1,IDN-Jakarta_ADM-4-union_1,ADM-4-union,IDN-Jakarta,IDN-Jakarta,2022-06-27,"MULTIPOLYGON (((106.78141 -6.31616, 106.78124 ..."


In [12]:
# loop over cities
for i in range(0,len(cities)):
    city = cities.loc[[i]]
    city_indicator = float(mean_tree_cover(city))
    cities.loc[i,"mean_tree_cover"] = city_indicator